# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Your thoughts here </font>

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> Your thoughts here </font>

### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to 
1. only include parcels within 1600 feet of the lakes we are studying, and 
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [2]:
# Lazily read the parcel dataset stored in the Delta table folder `data/parcel.delta`
# This avoids loading the entire dataset into memory; only the preview below is computed.
from pathlib import Path
import polars as pl

table_dir = Path('.data/parcel.delta')
if not table_dir.exists():
    raise FileNotFoundError(f"Delta table folder not found: {table_dir}")

# Try to use `deltalake` to list files if available (optional). If not available,
# fall back to scanning the folder for parquet files (skips the _delta_log).
parquet_files = []
try:
    from deltalake import DeltaTable
    dt = DeltaTable(str(table_dir))
    try:
        # dt.files() may return relative paths; make them absolute relative to table_dir
        parquet_files = [str(table_dir / f) if not Path(f).is_absolute() else str(f) for f in dt.files()]
    except Exception:
        parquet_files = []
except Exception:
    parquet_files = []

if not parquet_files:
    parquet_files = [str(p) for p in table_dir.rglob('*.parquet') if '_delta_log' not in str(p)]

if not parquet_files:
    raise RuntimeError(f'No parquet files found under {table_dir} — cannot create lazy scan')

print(f'Found {len(parquet_files)} parquet files — creating a lazy Polars scan (no data loaded yet)')

# Create a lazy Polars DataFrame from the Parquet files (deferred computation)
parcel_lazy = pl.scan_parquet(parquet_files)

# Display column names and a small preview (first 10 rows). The preview triggers a small computation only.
print('Columns:', list(parcel_lazy.schema.keys()))
parcel_lazy.limit(10).collect()


C:\Users\jc8396du\AppData\Local\Temp\ipykernel_27960\2286917183.py:18: DeprecationWarning: Call to deprecated method files. (Not compatible with modern delta features (e.g. shallow clones). Use `file_uris` instead.) -- Deprecated since version 1.0.0.
  parquet_files = [str(table_dir / f) if not Path(f).is_absolute() else str(f) for f in dt.files()]


Found 14755 parquet files — creating a lazy Polars scan (no data loaded yet)


C:\Users\jc8396du\AppData\Local\Temp\ipykernel_27960\2286917183.py:36: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print('Columns:', list(parcel_lazy.schema.keys()))


Columns: ['ACRES_DEED', 'ACRES_POLY', 'AGPRE_ENRD', 'AGPRE_EXPD', 'AG_PRESERV', 'BASEMENT', 'BLDG_NUM', 'BLOCK', 'CITY', 'CITY_USPS', 'COOLING', 'COUNTY_ID', 'DWELL_TYPE', 'EMV_BLDG', 'EMV_LAND', 'EMV_TOTAL', 'FIN_SQ_FT', 'GARAGE', 'GARAGESQFT', 'GREEN_ACRE', 'HEATING', 'HOMESTEAD', 'HOME_STYLE', 'LANDMARK', 'LOT', 'MULTI_USES', 'NUM_UNITS', 'OPEN_SPACE', 'OWNER_MORE', 'OWNER_NAME', 'OWN_ADD_L1', 'OWN_ADD_L2', 'OWN_ADD_L3', 'PARC_CODE', 'PIN', 'PLAT_NAME', 'PREFIXTYPE', 'PREFIX_DIR', 'SALE_DATE', 'SALE_VALUE', 'SCHOOL_DST', 'SPEC_ASSES', 'STREETNAME', 'STREETTYPE', 'SUFFIX_DIR', 'Shape_Area', 'Shape_Leng', 'TAX_ADD_L1', 'TAX_ADD_L2', 'TAX_ADD_L3', 'TAX_CAPAC', 'TAX_EXEMPT', 'TAX_NAME', 'TOTAL_TAX', 'UNIT_INFO', 'USE1_DESC', 'USE2_DESC', 'USE3_DESC', 'USE4_DESC', 'WSHD_DIST', 'XUSE1_DESC', 'XUSE2_DESC', 'XUSE3_DESC', 'XUSE4_DESC', 'YEAR_BUILT', 'Year', 'ZIP', 'ZIP4', 'centroid_lat', 'centroid_long', 'Distance_Parcel_Lake_meters']


ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,LANDMARK,LOT,MULTI_USES,NUM_UNITS,OPEN_SPACE,OWNER_MORE,OWNER_NAME,OWN_ADD_L1,OWN_ADD_L2,OWN_ADD_L3,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long,Distance_Parcel_Lake_meters
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64
"""0.0""","""0.42""",null,null,"""N""","""N""","""7878""","""003""","""MAPLE GROVE""","""MAPLE GROVE""","""1""","""053""",null,"""246300.0""","""306000.0""","""552300.0""","""2117.0""","""Y""","""600""","""N""","""2""","""Y""","""O""",null,"""002""",null,null,"""N""",null,"""P J & J A HENNEN TRUSTEES""",null,null,null,"""0.0""","""053-2111922440001""","""MAPLE GROVES LAKESHORE WEST 5T…",null,null,"""1992-05-01""","""225000.0""","""279""","""301.0""","""ITHACA LA N""",null,null,null,null,"""7878 ITHACA LA N""","""MAPLE GROVE MN 55311""",null,"""5654.0""","""N""","""PAUL J & JULIE A HENNEN""","""8898.0""",null,"""Residential Lakeshore""",null,null,null,"""Lower Minnesota River""",null,null,null,null,"""1983.0""","""2015""","""55311""",null,"""45.09821""","""-93.46463""",42.134976
"""0.0""","""0.23""",null,null,"""N""","""N""","""7720""","""001""","""MAPLE GROVE""","""MAPLE GROVE""","""1""","""053""",null,"""156400.0""","""70000.0""","""226400.0""","""1212.0""","""Y""","""516""","""N""","""2""","""Y""","""R""",null,"""002""",null,null,"""N""",null,"""D L DANIELSON/E J DANIELSON""",null,null,null,"""0.0""","""053-2111922340002""","""SHORECREST""",null,null,"""2003-04-01""","""260000.0""","""279""","""304.0""","""NIAGARA LA N""",null,null,null,null,"""7720 NIAGARA LA N""","""MAPLE GROVE MN 55311""",null,"""2095.0""","""N""","""DAVID & EARLENE DANIELSON""","""3553.0""",null,"""Residential""",null,null,null,"""Lower Minnesota River""",null,null,null,null,"""1979.0""","""2015""","""55311""",null,"""45.09521""","""-93.47206""",430.687512
"""0.0""","""0.31""",null,null,"""N""","""Y""","""8076""","""001""","""MAPLE GROVE""","""MAPLE GROVE""","""1""","""053""",null,"""135600.0""","""214000.0""","""349600.0""","""1588.0""","""Y""","""772""","""N""","""2""","""Y""","""R""",null,"""001""",null,null,"""N""",null,"""J W RANDALL & G M RANDALL""",null,null,null,"""0.0""","""053-2111922410001""","""MAPLE GROVES LAKESHORE WEST 5T…",null,null,"""2005-07-01""","""549000.0""","""279""","""0.0""","""ITHACA LA N""",null,null,"""1263.60751422""","""146.53037347""","""8076 ITHACA LA N""","""MAPLE GROVE MN 55311""",null,"""3438.0""","""N""","""JOSEPH & GRETCHEN RANDALL""","""5268.0""",null,"""Residential Lakeshore""",null,null,null,"""Lower Minnesota River""",null,null,null,null,"""1982.0""","""2015""","""55311""",null,"""45.10148""","""-93.46586""",27.255853
"""0.0""","""0.31""",null,null,"""N""","""Y""","""8076""","""001""","""MAPLE GROVE""","""MAPLE GROVE""","""1""","""053""",null,"""135600.0""","""214000.0""","""349600.0""","""1588.0""","""Y""","""772""","""N""","""2""","""Y""","""R""",null,"""001""",null,null,"""N""",null,"""J W RANDALL & G M RANDALL""",null,null,null,"""0.0""","""053-2111922410001""","""MAPLE GROVES LAKESHORE WEST 5T…",null,null,"""2005-07-01""","""549000.0""","""279""","""0.0""","""ITHACA LA N""",null,null,null,null,"""8076 ITHACA LA N""","""MAPLE GROVE MN 55311""",null,"""3438.0""","""N""","""JOSEPH & GRETCHEN RANDALL""","""5268.0""",null,"""Resi

## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.** 

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* DWELL_TYPE: Dwelling Type 
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [ ]:
# Your code here

## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [ ]:
# Your code here.

## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.

In [ ]:
# Your code here.